
## Biostat/Biomath M257 Homework 4
### Due May 12 @ 11:59PM'
#### Sisi Shao 205645669
#### 2023/05/12

System information (for reproducibility):

In [4]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.5.0)
  CPU: 10 × Apple M1 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, apple-m1)
  Threads: 1 on 8 virtual cores


Load packages:

In [5]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Downloads/SSS/Courses/Bio257_23Spring/HW/hw4`


Status `~/Downloads/SSS/Courses/Bio257_23Spring/HW/hw4/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [944b1d66] CodecZlib v0.7.1
  [0b1a1467] KrylovKit v0.6.0
  [bdcacae8] LoopVectorization v0.12.159
  [b51810bb] MatrixDepot v1.0.10
  [295af30f] Revise v3.5.2
  [2913bbd2] StatsBase v0.34.0
  [b8865327] UnicodePlots v3.5.3
  [8bb1440f] DelimitedFiles
  [37e2e46d] LinearAlgebra
  [2f01184e] SparseArrays


We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

## Q1 (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

## Sol Q1

The transition matrix $\mathbf{P}$ can be written as
\begin{align}
\mathbf{P}&=p\times\text{Diag}(\mathbf{A1})^{-1}\mathbf{A} + (\mathbf{1}-p\mathbb{1}_{\{\mathbf{A1}>\mathbf{0}\}})\odot\frac{\mathbf{11}^T}{n}
\end{align}
where 

- $\mathbf{A}$ is the **SPARSE** connectivity matrix.
- $\mathbf{1}$ is a $n\times 1$ vector with all ones.
- $\text{Diag}(\mathbf{A1})^{-1}$ is a diagonal matrix with elements $\frac{1}{r_i}=\frac{1}{\sum_{j=1}^na_{ij}}$.
- $\mathbb{1}_{\{\mathbf{A1}>\mathbf{0}\}}$ is the indicator vector such that if $r_i=0$, then the element equals 0.
- $\odot$ is the Hadamard product.

If $r_i=\sum_{j=1}a_{ij}=0$, then set the inverse equal to 0 as well.

Denote $\mathbf{W}=\text{Diag}(\mathbf{A1})^{-1}\mathbf{A}$ and $\mathbf{c}=\frac{1}{n}\left(\mathbf{1}-p\mathbb{1}_{\{\mathbf{A1}>\mathbf{0}\}}\right)\odot\mathbf{1}$, then the transition matrix is
\begin{align}
\mathbf{P}&=p\mathbf{W}+\mathbf{c}\mathbf{1}^T
\end{align}

where $\mathbf{W}$ is sparse and $\mathbf{c}\mathbf{1}^T$ is of rank one.

## Q2 Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.

## Sol Q2

To show proof $\mathbf{I-P^T}$ is rank deficient, it is enough to show that there exists a unique nonnegative solution $\mathbf{x}$ to

$$\mathbf{x}^T=\mathbf{x}^T\mathbf{P}$$

and $\mathbf{1}^T\mathbf{x}<\infty$. The reason is: $x=\mathbf{0}$ is always a solution to $\mathbf{x}^T=\mathbf{x}^T\mathbf{P}$ and full rank property of $\mathbf{I-P^T}$ implies that $\mathbf{0}$ is the **ONLY** solution, which is a contradiction to the statement that there exists another nonnegative solution.

Here I quote a theorem from Biostat 270:

**Theorem** ( Biostat 270, modified ) : Let $\{X_n:n\ge 0\}$ be a Markov chain with state space $\left(\mathbb{S},\mathcal{B}(\mathbb{S})\right)$ where $\mathbb{S}$ is countable. Also, let $\mathbf{P}$ be the transition matrix (or transition kernel) associated with the chain.

Assume the following:
- $\{X_n:n\ge 0\}$ is recurrent.
- $\mathbf{P}$ is irreducible.
- $0\in\mathbb{S}$ is a fixed state.
Then the linear system
$$\mathbf{x}^T=\mathbf{x}^T\mathbf{P}$$
has a **UNIQUE** nonnegative solution satisfying $x_0=1$. In the special case where $\mathbb{S}$ is finite, we have
$$\mathbf{x}^T\mathbf{1}<\infty$$
which means the solution vector $\mathbf{x}$ can be re-scaled to a probability vector s.t.
$$\mathbf{x}^T\mathbf{1}=\mathbf{1}.$$

## Q3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [6]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

[ Info: verify download of index files...
[ Info: reading database
[ Info: adding metadata...
[ Info: adding svd data...
[ Info: writing database
[ Info: used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [7]:
# connectivity matrix
A = md.A

916428×916428 SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

Compute summary statistics:  

* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  

* number of web pages

* number of edges (web links)

* number of dangling nodes (pages with no out links)

* histogram of in-degrees  

* list the top 20 pages with the largest in-degrees?  

* histogram of out-degrees

* which the top 20 pages with the largest out-degrees?

* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

**Hint**: For plots, you can use the [UnicodePlots.jl](https://github.com/Evizero/UnicodePlots.jl) package (`spy`, `histogram`, etc), which is fast for large data. 

## Sol Q3

- How much memory does `A` take?

**Sol**: According to the table returned by `varinfo()`, it takes 50.809 MB.

In [8]:
varinfo()

| name         |       size | summary                                                                              |
|:------------ | ----------:|:------------------------------------------------------------------------------------ |
| A            | 50.809 MiB | 916428×916428 SparseMatrixCSC{Bool, Int64} with 5105039 stored entries               |
| Base         |            | Module                                                                               |
| Core         |            | Module                                                                               |
| Main         |            | Module                                                                               |
| PageRankImPt |  120 bytes | UnionAll                                                                             |
| md           | 50.811 MiB | MatrixDepot.MatrixDescriptor{MatrixDepot.RemoteMatrixData{MatrixDepot.SSRemoteType}} |
| xsol         |  6.992 MiB | 916428-element Vector{Float64}                                                       |


- If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take? 

**Sol**: All elements will be converted from `Bool` to `Float64` and all zeros will be filled, so it will take (approximately)

$$m=8\times col(A)\times row(A)\ /\ 1024\ (KB)$$

After some calculation, 
$$m=8\times 916428 \times 916428\ /\ 1024\approx 6.56\times 10^{9} KB$$
or $$6.41\times 10^{3}\ GB$$ 
equivalently.

- number of web pages

**Sol**: 916428.

- number of edges (web links)

**Sol**: Indeed this should be 5105039 edges.

- number of dangling nodes (pages with no out links)

**Sol**:

In [9]:
using SparseArrays

B = findnz(A)

916428 - length(unique(B[1]))

176974

- histogram of in-degrees where $in\_degrees=\sum_ia_{ij}$.

In [10]:
using UnicodePlots
using StatsBase

in_degrees = collect(values(countmap(B[2]))) 
histogram(in_degrees, nbins = 35, closed = :left, xscale=log10)

[ Info: Precompiling UnicodePlots [b8865327-cd53-5732-bb35-84acbb429228]


                    ┌                                        ┐ 
   [   0.0,  200.0) ┤███████████████████████████████  713 286  
   [ 200.0,  400.0) ┤███████████████▍ 812                      
   [ 400.0,  600.0) ┤████████████▎ 203                         
   [ 600.0,  800.0) ┤██████████▏ 80                            
   [ 800.0, 1000.0) ┤███████▊ 30                               
   [1000.0, 1200.0) ┤██████▊ 19                                
   [1200.0, 1400.0) ┤█████▋ 12                                 
   [1400.0, 1600.0) ┤████▎ 6                                   
   [1600.0, 1800.0) ┤████▎ 6                                   
   [1800.0, 2000.0) ┤█████▏ 9                                  
   [2000.0, 2200.0) ┤████▊ 8                                   
   [2200.0, 2400.0) ┤███▎ 4                                    
   [2400.0, 2600.0) ┤████▌ 7                                   
   [2600.0, 2800.0) ┤█████▌ 11                                 
   [2800.0, 3000.0) ┤████▎ 6            

In [11]:
sort(countmap(B[2]), byvalue=true, rev=true)

OrderedCollections.OrderedDict{Int64, Int64} with 714545 entries:
  537040 => 6326
  597622 => 5354
  504141 => 5271
  751385 => 5182
  32164  => 5097
  885606 => 4847
  163076 => 4731
  819224 => 4620
  605857 => 4550
  828964 => 4484
  551830 => 4220
  41910  => 4219
  558792 => 4206
  459075 => 4187
  407611 => 4180
  213433 => 4084
  765335 => 4015
  384667 => 4010
  173977 => 3988
  687326 => 3956
  425771 => 3940
  396322 => 3913
  486981 => 3908
  182122 => 3872
  908352 => 3810
  ⋮      => ⋮

- histogram of out-degrees

$out\_degrees=\sum_{j=1}^na_{ij}$

In [12]:
out_degrees = collect(values(countmap(B[1]))) 
histogram(out_degrees, nbins = 35, closed = :left, xscale=log10)

                  ┌                                        ┐ 
   [  0.0,  20.0) ┤███████████████████████████████  714 824  
   [ 20.0,  40.0) ┤███████████████████████▏ 22 628           
   [ 40.0,  60.0) ┤████████████████▌ 1 329                   
   [ 60.0,  80.0) ┤█████████████▋ 371                        
   [ 80.0, 100.0) ┤███████████▎ 124                          
   [100.0, 120.0) ┤██████████▎ 86                            
   [120.0, 140.0) ┤███████▋ 29                               
   [140.0, 160.0) ┤██████▎ 15                                
   [160.0, 180.0) ┤██████▎ 15                                
   [180.0, 200.0) ┤████▊ 8                                   
   [200.0, 220.0) ┤████▌ 7                                   
   [220.0, 240.0) ┤██▌ 3                                     
   [240.0, 260.0) ┤████▊ 8                                   
   [260.0, 280.0) ┤██▌ 3                                     
   [280.0, 300.0) ┤  0                                       
   [300.

- which the top 20 pages with the largest out-degrees?

In [13]:
sort(countmap(B[1]), byvalue=true, rev=true)

OrderedCollections.OrderedDict{Int64, Int64} with 739454 entries:
  506743 => 456
  203749 => 372
  305230 => 372
  768092 => 330
  808644 => 277
  412411 => 268
  600480 => 265
  376429 => 258
  156951 => 257
  885729 => 256
  667585 => 253
  685696 => 248
  282141 => 247
  598189 => 245
  579315 => 244
  411594 => 231
  321092 => 229
  838279 => 225
  302734 => 216
  915274 => 213
  285815 => 210
  41298  => 206
  432907 => 206
  440090 => 203
  674718 => 200
  ⋮      => ⋮

- visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 


In [14]:
spy(A[1:10000, 1:10000])

          ┌──────────────────────────────────────────┐    
        1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
          │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
          │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
          │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
          │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
          │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
          │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
          │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
          │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
          │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
          │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
          │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
          │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠀⠁⠀⠀⠀│    
          │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
          │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠁⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
          │⠈⠀⠀⠄⠀⠀⡀⠈⠀⠀⠙⠀⠀⠀⢀⢀⠂⠀⠀⡄⢀⠀⠠⠀⠂⠀⠀⠐⠀⠀⠸⠀⠀⠀⠈⠠⠀⠠⠁⠀⠂⠀│  

## Q4 (5 pts) Dense linear algebra? 

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer. 

## Sol Q4

In [15]:
using BenchmarkTools, LinearAlgebra

function test_lu(Σ::Matrix)
   lu(Σ)
end

Σ = randn(1000, 1000)

@benchmark test_lu($Σ)

BenchmarkTools.Trial: 392 samples with 1 evaluation.
 Range (min … max):   4.065 ms … 37.635 ms  ┊ GC (min … max): 0.00% … 27.80%
 Time  (median):     11.700 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.763 ms ±  4.372 ms  ┊ GC (mean ± σ):  2.21% ±  7.63%

                ▁█▁▄▂  ▁                                       
  ▄▃▃▁▁▃▃▃▂▃▄▃▅▆█████▇▆█▇▄▇▆▄▄▃▄▄▃▃▄▃▃▃▃▂▄▂▃▃▃▁▃▂▁▁▁▂▂▁▁▁▂▁▁▂ ▃
  4.07 ms         Histogram: frequency by time        28.5 ms <

 Memory estimate: 7.64 MiB, allocs estimate: 3.

- (1) A quote from the GELU lecture: "To solve  𝐀𝐱=𝐛  via LU, we need a total of $\frac{2}{3}n^3+2n^2$ flops." Besides, the whole LU algorithm is done in place and we need a matrix to store $\mathbf{L}$ or $\mathbf{M}_i,i=1,\cdots,n-1$ and get $\mathbf{U}$ automatically.

Thus, to estimate the memory usage, we need approximately
$$2\times8\times916428^2 / 1024^3 = 12514.595\ GB$$

Besides, according to the results given by benchmark, 
$$7.64 / 1000^2 \times 916428^2 / 1024 \approx 6265.996\ GB$$
which means the benchmark function doesn't take the additional matrix into account.

- (2) By the $O(n^3)$ result, it will take approximately
$$18.981 / 1000^3 \times 916428^3/1000/3600\approx 4057.996\ h$$
to finish the LU decomposition.

## Q5 (75 pts) Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide are methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

In [16]:
using BenchmarkTools, LinearAlgebra, SparseArrays, Revise

# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    n         :: IA # nrow and ncol of A
    storage_n :: Matrix{IA} # an intermediate nx1 matrix
    one       :: Vector{T} # a nx1 vector with elements corresponding to \mathbf{c}
    degree    :: Vector{T} # inverse of the degree vector
    E         :: Diagonal{T} # Inverse of degree matrix
    W         :: SparseMatrixCSC{T, IA} # normalized A times telep
    Wt        :: Transpose{T, SparseMatrixCSC{T, IA}} # transpose of W
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n = size(A, 1)
    # TODO: initialize and pre-allocate arrays
    
    degree = Vector{T}(undef, n)
    one = Vector{T}(undef, n)
    
    storage_n = sum(A, dims=2)
    
    # I am trying not to use for loop for simplicity
    #for i in 1:n
    #    if storage_n[i] > 0.0
    #        degree[i] = telep / storage_n[i]
    #        one[i] = (1 - telep) / n
    #    elseif storage_n[i] == 0.0
    #        degree[i] = 0.0
    #        one[i] = 1 / n
    #    end
    #end
    
    one .= (1 .- (vec(storage_n) .> 0) .* telep) / n
    E = Diagonal(replace(1 ./ vec(storage_n) .* telep, Inf=>0))

    #E = Diagonal(degree)
    W = convert(SparseMatrixCSC{Float64, Int64}, A)
    mul!(W, E, A)
    Wt = transpose(W)    
    
    PageRankImPt(A, telep, n, storage_n, one, degree, E, W, Wt)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)
# TODO: implement this function for evaluating M[i, j]
#Base.getindex(M::PageRankImPt, i, j) = M.telep

###################################################################
#####                         IMPORTANT                       #####
##### The following function is suggested by Elvis Han Cui.   #####
###################################################################

function Base.getindex(M::PageRankImPt, i, j)
    if i == j
        out = 1 - M.A[j, i] * M.degree[i] - M.one[j]
    else
                out = - M.A[j, i] * M.degree[i] - M.one[j]
    end
    return out
end 

# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}) where T <: AbstractFloat
    # TODO: implement mul!(out, M, v)
    #sleep(1e-2) # wait 10 ms as if your code takes 1ms
    
    M.degree .= dot(M.one, v)
    mul!(out, M.Wt, v)
    axpby!(1, M.degree, 1, out)
    axpby!(1, v, -1, out)
    
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    # TODO: implement mul!(out, transpose(M), v)
    #sleep(1e-2) # wait 10 ms as if your code takes 1ms
    
    mul!(out, M.W, v)
    axpby!(sum(v), M.one, 1, out)
    axpy!(-1, v, out)
    
    out
end

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M} \mathbf{x} = \mathbf{0}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv.gz`. **Do not put this file in your Git**. You will lose points if you do. You can add a line `pgrksol.csv.gz` to your `.gitignore` file.

In [17]:
using CodecZlib, DelimitedFiles

#isfile("pgrksol.csv.gz") || download("https://raw.githubusercontent.com/ucla-biostat-257/2023spring/master/hw/hw4/pgrksol.csv.gz")
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

In [18]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

#@assert transpose(M) * ones(n) ≈ zeros(n)
@assert norm(transpose(M) * ones(n)) < 1e-12

In [19]:
#@assert M * xsol ≈ zeros(n)
@assert norm(M * xsol) < 1e-12

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate no memory.

In [20]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 406 samples with 1 evaluation.
 Range (min … max):  12.118 ms … 12.510 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.154 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.164 ms ± 35.888 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▁▃▆▆▆██▆                                                
  ▂▁▁▄▅█████████▇▆█▆▆█▇▄▄▃▄▂▄▂▂▂▂▂▃▃▂▂▁▃▃▁▁▂▂▂▁▁▄▁▂▁▁▂▁▁▁▁▁▁▂ ▄
  12.1 ms         Histogram: frequency by time        12.3 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [21]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 324 samples with 1 evaluation.
 Range (min … max):  12.829 ms … 44.676 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     13.834 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.219 ms ±  3.867 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▇▅▅▃▁▂▂▁▁▂ ▁                                                
  ███████████▆██▇▆▆▇▄▅▅▅▅▄▄▄▄▅▄▁▁▄▄▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▄▁▁▁▁▁▁▁▁▁▄ ▆
  12.8 ms      Histogram: log(frequency) by time      34.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [22]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

20.0

**Hint**: My median run times are about 10 ms and memory allocations are 0 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [23]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(value = ([3.378342822237397e-5, 2.071015539278128e-6, 3.6630659852902857e-6, 7.527510785714568e-7, 8.633285997289008e-7, 1.7694182527638176e-6, 2.431230382949412e-7, 6.368417180841316e-7, 4.7449737038222246e-7, 2.689548611137735e-7  …  3.167946974023205e-6, 9.726880410372266e-7, 6.588614478640728e-7, 7.737011774829107e-7, 2.431230382949412e-7, 1.6219204214483919e-6, 3.9121300606903596e-7, 2.431230382949412e-7, 7.296033831434173e-6, 6.330939996774085e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821116321355378e-13,).
), time = 4.308896458, bytes = 1008962679, gctime = 0.026892125, gcstats = Base.GC_Diff(1008962679, 137, 0, 79420, 3, 113, 26892125, 2, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [24]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is

In [25]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 3-4 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [26]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

(value = (ComplexF64[-1.6565663346282113e-13 + 0.0im], Vector{ComplexF64}[[0.0056358269538065026 + 0.0im, 0.00034549143807822025 + 0.0im, 0.0006110808494125085 + 0.0im, 0.00012557561626038905 + 0.0im, 0.00014402240532806492 + 0.0im, 0.0002951783050398909 + 0.0im, 4.0558328286807705e-5 + 0.0im, 0.000106239357848543 + 0.0im, 7.915671116200243e-5 + 0.0im, 4.486765066728042e-5 + 0.0im  …  0.0005284839899796172 + 0.0im, 0.00016226599147987382 + 0.0im, 0.00010991273837613301 + 0.0im, 0.000129070558557546 + 0.0im, 4.0558328286807705e-5 + 0.0im, 0.00027057238743206593 + 0.0im, 6.526302748336464e-5 + 0.0im, 4.0558328286807705e-5 + 0.0im, 0.0012171406601332844 + 0.0im, 0.0001056141551070456 + 0.0im]], ConvergenceInfo: one converged value after 7 iterations and 99 applications of the linear map;
norms of residuals are given by (9.574656910926385e-14,).
), time = 8.978559708, bytes = 1579133070, gctime = 0.05488225, gcstats = Base.GC_Diff(1579133070, 213, 0, 42748, 13, 217, 54882250, 4, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [27]:
@assert abs(Real(vals[1])) < 1e-8

In [28]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

Arnoldi should be reasonably fast. The points you'll get is

In [29]:
clamp(20 / time_arnoldi * 20, 0, 20)

20.0

**Hint**: My runtime is about 6-7 seconds.

## Q6 (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees? 

## Sol 6

- The results are identical.

In [30]:
index = Vector(1:916428)
score_vec= Dict(index .=> x0)
sort(score_vec, byvalue=true, rev=true)

OrderedCollections.OrderedDict{Int64, Float64} with 916428 entries:
  537040 => 0.00105074
  597622 => 0.000889318
  504141 => 0.000875534
  751385 => 0.000860754
  32164  => 0.000846638
  885606 => 0.000805119
  163076 => 0.000785855
  819224 => 0.000767421
  605857 => 0.000755796
  828964 => 0.000744835
  551830 => 0.000700992
  41910  => 0.000700826
  558792 => 0.000698667
  459075 => 0.000695512
  407611 => 0.000694349
  213433 => 0.000678406
  765335 => 0.000666947
  384667 => 0.000666117
  173977 => 0.000662463
  687326 => 0.000657149
  425771 => 0.000654492
  396322 => 0.000650008
  486981 => 0.000649177
  182122 => 0.000643199
  908352 => 0.000632902
  ⋮      => ⋮

In [31]:
sort(countmap(B[2]), byvalue=true, rev=true)

OrderedCollections.OrderedDict{Int64, Int64} with 714545 entries:
  537040 => 6326
  597622 => 5354
  504141 => 5271
  751385 => 5182
  32164  => 5097
  885606 => 4847
  163076 => 4731
  819224 => 4620
  605857 => 4550
  828964 => 4484
  551830 => 4220
  41910  => 4219
  558792 => 4206
  459075 => 4187
  407611 => 4180
  213433 => 4084
  765335 => 4015
  384667 => 4010
  173977 => 3988
  687326 => 3956
  425771 => 3940
  396322 => 3913
  486981 => 3908
  182122 => 3872
  908352 => 3810
  ⋮      => ⋮

## Q7 Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.